do not delete anything annessesary for this adjustment, make everything connected when whatever i input in here {USER_COUNTRY = "Philippines"  # red
USER_COLOR = "red"

HIGHLIGHT_COUNTRY = "Japan"  # orange
HIGHLIGHT_COLOR = "orange"}, make the dataframe connected to that, make the narrative conected to that